<a href="https://colab.research.google.com/github/rahmanziaur/DTClassifierTest/blob/main/R_20_09_22_one-hot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# evaluate logistic regression on the breast cancer dataset with an one-hot encoding
import timeit
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
# define the location of the dataset
#from numpy import argmax as ag
# url = "https://raw.githubusercontent.com/rahmanziaur/DTClassifierTest/main/diabetic_data - Copy.csv"

# Rubel 
# url = "/content/drive/MyDrive/0_ColabDatasets/final Medical.csv"


# load the dataset
#dataset = pd.read_csv(url, header=None)
# retrieve the array of data

# Preprocessed Numeric Data
# dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/Processed3_TestDiabData_excepDiag.csv')

# Original raw Data
dataset=pd.read_csv('https://raw.githubusercontent.com/rahmanziaur/DTClassifierTest/main/diabetic_data_exceptDiag.csv')

# dropping 'gender' because object to int type change issue
# not required if not using exported .csv dataset 

# del dataset[dataset.columns[0]]

# moving the target column at the end. No need for Rubel
column_to_move = dataset.pop('readmitted')
dataset.insert(len(dataset.columns), 'readmitted', column_to_move)



data = dataset.values
# separate into input and output columns
X = data[:, :-1].astype(str)
y = data[:, -1].astype(str)
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# one-hot encode input variables
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoder.fit(X_train)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
label_encoder.fit(y_test)
y_test = label_encoder.transform(y_test)
# define the model
clf = RandomForestClassifier()
# fit on the training set
clf.fit(X_train, y_train)
# predict on test set
yhat = clf.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100),'%')

Accuracy: 58.13 %


In [11]:
#dataset['age'] = dataset['age'].astype('int64')
#dataset['age'] = dataset['age'].astype('int')
print(dataset.age.value_counts())
# convert age categories to mid-point values
age_dict = {1:5, 2:15, 3:25, 4:35, 5:45, 6:55, 7:65, 8:75, 9:85, 10:95}
dataset['age'] = dataset.age.map(age_dict)
print(dataset.age.value_counts())



[70-80)     26068
[60-70)     22483
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)     2793
[20-30)      1657
[10-20)       691
[0-10)        161
Name: age, dtype: int64
Series([], Name: age, dtype: int64)


In [ ]:
# convert data type of nominal features in dataframe to 'object' type
i = ['encounter_id', 'patient_nbr', 'gender', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',\
          'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', \
          'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose','miglitol', \
          'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', \
          'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', \
          'age', 'A1Cresult', 'max_glu_serum',]

dataset[i]

In [23]:
num_col = list(set(list(dataset._get_numeric_data().columns))- {'readmitted'})
num_col


['patient_nbr',
 'num_procedures',
 'number_outpatient',
 'nummed',
 'num_lab_procedures',
 'admission_type_id',
 'discharge_disposition_id',
 'number_inpatient',
 'admission_source_id',
 'num_medications',
 'number_emergency',
 'age',
 'encounter_id',
 'number_diagnoses',
 'time_in_hospital']

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Removing skewnewss and kurtosis using log transformation if it is above a threshold value -  2

statdataframe = pd.DataFrame()
statdataframe['numeric_column'] = num_col
skew_before = []
skew_after = []

kurt_before = []
kurt_after = []

standard_deviation_before = []
standard_deviation_after = []

log_transform_needed = []

log_type = []

for i in num_col:
    skewval = dataset[i].skew()
    skew_before.append(skewval)
    
    kurtval = dataset[i].kurtosis()
    kurt_before.append(kurtval)
    
    sdval = dataset[i].std()
    standard_deviation_before.append(sdval)
    
    if (abs(skewval) >2) & (abs(kurtval) >2):
        log_transform_needed.append('Yes')
        
        if len(dataset[dataset[i] == 0])/len(dataset) <=0.02:
            log_type.append('log')
            # skewvalnew = np.log(pd.DataFrame(dataset[train_data[i] > 0])[i]).skew()
            # skew_after.append(skewvalnew)
            
            # kurtvalnew = np.log(pd.DataFrame(dataset[train_data[i] > 0])[i]).kurtosis()
            # kurt_after.append(kurtvalnew)
            
            # sdvalnew = np.log(pd.DataFrame(dataset[train_data[i] > 0])[i]).std()
            # standard_deviation_after.append(sdvalnew)
            
        else:
            log_type.append('log1p')
            skewvalnew = np.log1p(pd.DataFrame(dataset[dataset[i] >= 0])[i]).skew()
            skew_after.append(skewvalnew)
        
            kurtvalnew = np.log1p(pd.DataFrame(dataset[dataset[i] >= 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log1p(pd.DataFrame(dataset[dataset[i] >= 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
    else:
        log_type.append('NA')
        log_transform_needed.append('No')
        
        skew_after.append(skewval)
        kurt_after.append(kurtval)
        standard_deviation_after.append(sdval)

statdataframe['skew_before'] = skew_before
statdataframe['kurtosis_before'] = kurt_before
statdataframe['standard_deviation_before'] = standard_deviation_before
statdataframe['log_transform_needed'] = log_transform_needed
statdataframe['log_type'] = log_type
# statdataframe['skew_after'] = skew_after
# statdataframe['kurtosis_after'] = kurt_after
# statdataframe['standard_deviation_after'] = standard_deviation_after



In [30]:
statdataframe

,numeric_column,skew_before,kurtosis_before,standard_deviation_before,log_transform_needed,log_type
0,patient_nbr,0.471281,-0.347372,3.869636e+07,No,NA
1,num_procedures,1.316415,0.857110,1.705807e+00,No,NA
2,number_outpatient,8.832959,147.907736,1.267265e+00,Yes,log1p
3,nummed,0.000000,0.000000,0.000000e+00,No,NA
4,num_lab_procedures,-0.236544,-0.245074,1.967436e+01,No,NA
5,admission_type_id,1.591984,1.942476,1.445403e+00,No,NA
6,discharge_disposition_id,2.563067,6.003347,5.280166e+00,Yes,log
7,number_inpatient,3.614139,20.719397,1.262863e+00,Yes,log1p
8,admission_source_id,1.029935,1.744989,4.064081e+00,No,NA
9,num_medications,1.326672,3.468155,8.127566e+00,No,NA


In [31]:
for i in range(len(statdataframe)):
    if statdataframe['log_transform_needed'][i] == 'Yes':
        colname = str(statdataframe['numeric_column'][i])
        
        if statdataframe['log_type'][i] == 'log':
            dataset = dataset[dataset[colname] > 0]
            dataset[colname + "_log"] = np.log(dataset[colname])
            
        elif statdataframe['log_type'][i] == 'log1p':
            dataset = dataset[dataset[colname] >= 0]
            dataset[colname + "_log1p"] = np.log1p(dataset[colname])



In [33]:
dataset = dataset.drop(['number_outpatient', 'number_inpatient', 'number_emergency'], axis = 1)



In [34]:
dataset.shape

(101766, 49)

In [36]:
# get list of only numeric features
numerics = list(set(list(dataset._get_numeric_data().columns))- {'readmitted'})
numerics

['patient_nbr',
 'num_procedures',
 'discharge_disposition_id_log',
 'discharge_disposition_id',
 'number_inpatient_log1p',
 'admission_source_id',
 'num_medications',
 'age',
 'encounter_id',
 'time_in_hospital',
 'nummed',
 'num_lab_procedures',
 'number_outpatient_log1p',
 'admission_type_id',
 'number_emergency_log1p',
 'number_diagnoses']

In [40]:
# show list of features that are categorical
dataset.encounter_id = dataset.encounter_id.astype('int64')
dataset.patient_nbr = dataset.patient_nbr.astype('int64')
#dataset.diabetesMed = dataset.diabetesMed.astype('int64')
#dataset.change = dataset.change.astype('int64')

# convert data type of nominal features in dataframe to 'object' type for aggregating
i = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', \
          'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose','miglitol', \
          'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', \
          'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone','A1Cresult']
#dataset[i] = dataset[i].astype('int64')

dataset.dtypes



encounter_id                      int64
patient_nbr                       int64
race                             object
gender                           object
age                             float64
weight                           object
admission_type_id                 int64
discharge_disposition_id          int64
admission_source_id               int64
time_in_hospital                  int64
payer_code                       object
medical_specialty                object
num_lab_procedures                int64
num_procedures                    int64
num_medications                   int64
number_diagnoses                  int64
max_glu_serum                    object
A1Cresult                        object
metformin                        object
repaglinide                      object
nateglinide                      object
chlorpropamide                   object
glimepiride                      object
acetohexamide                    object
glipizide                        object


In [41]:
dfcopy = dataset.copy(deep = True)

In [42]:
dataset['readmitted'] = dataset['readmitted'].apply(lambda x: 0 if x == 2 else x)

In [43]:
interactionterms = [('num_medications','time_in_hospital'),
('num_medications','num_procedures'),
('time_in_hospital','num_lab_procedures'),
('num_medications','num_lab_procedures'),
('num_medications','number_diagnoses'),
('age','number_diagnoses'),
('change','num_medications'),
('number_diagnoses','time_in_hospital'),
('num_medications','numchange')]

In [45]:
dataset[['num_medications','time_in_hospital', 'num_medications|time_in_hospital']].head()

,num_medications,time_in_hospital,num_medications|time_in_hospital
0,1,1,1
1,18,3,54
2,13,2,26
3,16,2,32
4,8,1,8


In [47]:
# Feature Scaling
datf = pd.DataFrame()
datf['features'] = numerics
datf['std_dev'] = datf['features'].apply(lambda x: dataset[x].std())
datf['mean'] = datf['features'].apply(lambda x: dataset[x].mean())

In [49]:
# dropping multiple encounters while keeping either first or last encounter of these patients
df2 = dataset.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
df2.shape

(71518, 57)

In [50]:
# standardize function
def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))

In [51]:
import scipy as sp

df2[numerics] = standardize(df2[numerics])

df2 = df2[(np.abs(sp.stats.zscore(df2[numerics])) < 3).all(axis=1)]

In [ ]:
from matplotlib.colors import ListedColormap
my_cmap = ListedColormap(sns.light_palette((250, 100, 50), input="husl", n_colors=50).as_hex())
table = df2.drop(['patient_nbr', 'encounter_id'], axis=1).corr(method='pearson')
table.style.background_gradient(cmap=my_cmap, axis = 0)

In [60]:
non_num_cols = ['race', 'gender', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 
                'max_glu_serum', 'A1Cresult', ]

In [55]:
num_cols = list(set(list(dataset._get_numeric_data().columns))- {'readmitted', 'change'})
num_cols

['patient_nbr',
 'num_procedures',
 'age|number_diagnoses',
 'time_in_hospital|num_lab_procedures',
 'discharge_disposition_id_log',
 'num_medications|num_lab_procedures',
 'num_medications|number_diagnoses',
 'discharge_disposition_id',
 'number_inpatient_log1p',
 'admission_source_id',
 'num_medications',
 'age',
 'num_medications|time_in_hospital',
 'encounter_id',
 'time_in_hospital',
 'number_diagnoses|time_in_hospital',
 'nummed',
 'num_lab_procedures',
 'number_outpatient_log1p',
 'admission_type_id',
 'num_medications|num_procedures',
 'number_emergency_log1p',
 'number_diagnoses']

In [56]:
feature_set = ['age', 'time_in_hospital', 'num_procedures', 'num_medications', 'number_outpatient_log1p', 
                 'number_emergency_log1p', 'number_inpatient_log1p', 'number_diagnoses', 'metformin', 
                 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide',
                 'pioglitazone', 'rosiglitazone', 'acarbose', 'tolazamide', 'insulin', 'glyburide-metformin',
                 'AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other', 'gender_1', 
                 'admission_type_id_3', 'admission_type_id_5', 'discharge_disposition_id_2', 'discharge_disposition_id_7', 
                 'discharge_disposition_id_10', 'discharge_disposition_id_18', 'admission_source_id_4',
                 'admission_source_id_7', 'admission_source_id_9', 'max_glu_serum_0', 'max_glu_serum_1', 'A1Cresult_0',
                 'A1Cresult_1', 'num_medications|time_in_hospital', 'num_medications|num_procedures',
                 'time_in_hospital|num_lab_procedures', 'num_medications|num_lab_procedures', 'num_medications|number_diagnoses',
                 'age|number_diagnoses', 'change|num_medications', 'number_diagnoses|time_in_hospital',
                 'num_medications|numchange']

In [3]:
# example of chi squared feature selection for categorical data
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from matplotlib import pyplot

# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=None)
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
  oe = OrdinalEncoder()
  oe.fit(X_train)
  X_train_enc = oe.transform(X_train)
  oe.fit(X_test)
  X_test_enc = oe.transform(X_test)
  return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# feature selection
def select_features(X_train, y_train, X_test):
	fs = SelectKBest(score_func=chi2, k='all')
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

# load the dataset
X, y = load_dataset('/content/drive/MyDrive/0_ColabDatasets/final Medical.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

TypeError: ignored

In [ ]:
# evaluation of a model fit using chi squared input features
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=None)
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
  oe = OrdinalEncoder()
  oe.fit(X_train)
  X_train_enc = oe.transform(X_train)
  oe.fit(X_test)
  X_test_enc = oe.transform(X_test)
  return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# feature selection
def select_features(X_train, y_train, X_test):
	fs = SelectKBest(score_func=chi2, k=30)
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs

# load the dataset
X, y = load_dataset('/content/drive/MyDrive/Colab Notebooks/Zia Sir/20-09-22/diabetic_data - Copy.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# feature selection
X_train_fs, X_test_fs = select_features(X_train_enc, y_train_enc, X_test_enc)
# fit the model
model = RandomForestClassifier()
model.fit(X_train_fs, y_train_enc)
# evaluate the model
yhat = model.predict(X_test_fs)
# evaluate predictions
accuracy = accuracy_score(y_test_enc, yhat)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 56.97
